In [1]:
import flair
flair.set_seed(2)

from flair.data import Corpus, Sentence
from flair.datasets import TREC_6, CSVClassificationCorpus
from flair.embeddings import TransformerDocumentEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer

import torch
import argparse
import json
import csv
import re
import time

import pandas as pd

In [2]:
learning_rate = 5.0e-5
mini_batch_size = 4
max_epochs = 10
model_name = "deepvk/USER-bge-m3"

In [3]:

        
column_name_map = {0: "text", 1: "label"}
corpus: Corpus = CSVClassificationCorpus("data/flair_data",
                                            column_name_map,
                                            skip_header=False,
                                            delimiter='\t',    # tab-separated files
                                            label_type='label')



label_dict = corpus.make_label_dictionary(label_type='label')

2024-11-16 19:24:45,923 Reading data from data\flair_data
2024-11-16 19:24:45,923 Train: data\flair_data\train.csv
2024-11-16 19:24:45,924 Dev: data\flair_data\dev.csv
2024-11-16 19:24:45,925 Test: data\flair_data\test.csv
2024-11-16 19:24:45,938 Computing label dictionary. Progress:


0it [00:00, ?it/s]
4068it [00:01, 3985.26it/s]

2024-11-16 19:24:46,967 Dictionary created for label 'label' with 9 values: FOOD_GOODS (seen 922 times), NON_FOOD_GOODS (seen 895 times), SERVICE (seen 884 times), LEASING (seen 380 times), LOAN (seen 380 times), REALE_STATE (seen 256 times), BANK_SERVICE (seen 215 times), NOT_CLASSIFIED (seen 117 times), TAX (seen 19 times)


In [4]:
document_embeddings = TransformerDocumentEmbeddings(model_name, fine_tune=True)
classifier = TextClassifier(document_embeddings, label_dictionary=label_dict, label_type='label')

trainer = ModelTrainer(classifier, corpus)

trainer.fine_tune(f'./models/gpt_data_v2/',
                    learning_rate=learning_rate,
                    mini_batch_size=mini_batch_size,
                    max_epochs=max_epochs,
                    monitor_test=True, train_with_dev=False, save_model_each_k_epochs=1)


2024-11-16 19:24:50,923 ----------------------------------------------------------------------------------------------------
2024-11-16 19:24:50,925 Model: "TextClassifier(
  (embeddings): TransformerDocumentEmbeddings(
    (model): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(46167, 1024, padding_idx=1)
        (position_embeddings): Embedding(8194, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x XLMRobertaLayer(
            (attention): XLMRobertaAttention(
              (self): XLMRobertaSdpaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linea

100%|██████████| 32/32 [00:09<00:00,  3.28it/s]

2024-11-16 19:31:56,601 DEV : loss 0.4902525246143341 - f1-score (micro avg)  0.922



100%|██████████| 32/32 [00:11<00:00,  2.76it/s]

2024-11-16 19:32:08,390 TEST : loss 0.4902525246143341 - f1-score (micro avg)  0.922


...

2024-11-16 20:52:42,518 DEV : loss 0.009175065904855728 - f1-score (micro avg)  0.998



100%|██████████| 32/32 [00:12<00:00,  2.51it/s]

2024-11-16 20:52:55,446 TEST : loss 0.009175065904855728 - f1-score (micro avg)  0.998


2024-11-16 20:52:55,598 ----------------------------------------------------------------------------------------------------
2024-11-16 20:53:59,663 epoch 10 - iter 101/1017 - loss 0.01194265 - time (sec): 64.06 - samples/sec: 6.31 - lr: 0.000005 - momentum: 0.000000
2024-11-16 20:54:43,432 epoch 10 - iter 202/1017 - loss 0.01586806 - time (sec): 107.83 - samples/sec: 7.49 - lr: 0.000004 - momentum: 0.000000
2024-11-16 20:55:33,748 epoch 10 - iter 303/1017 - loss 0.01074929 - time (sec): 158.15 - samples/sec: 7.66 - lr: 0.000004 - momentum: 0.000000
2024-11-16 20:56:25,789 epoch 10 - iter 404/1017 - loss 0.02239072 - time (sec): 210.19 - samples/sec: 7.69 - lr: 0.000003 - momentum: 0.000000
2024-11-16 20:57:20,297 epoch 10 - iter 505/1017 - loss 0.01797796 - time (sec): 264.70 - samples/sec: 7.63 - lr: 0.000003 - momentum: 0.000000
2024-11-16 20:58:13,356 epoch 10 - iter 606/1017 - loss 0.01499590 - time (sec): 317.76 - samples/sec: 7.63 - lr: 0.000002 - momentum: 0.000000
2024-11-16 2

RuntimeError: [enforce fail at inline_container.cc:424] . unexpected pos 3408256 vs 3408150

In [2]:
model = TextClassifier.load("models/gpt_data/model_epoch_5.pt")

In [4]:
result = model.evaluate(corpus.test, gold_label_type="label", mini_batch_size=2)

100%|██████████| 250/250 [00:18<00:00, 13.66it/s]


In [11]:
sentences = [sentence for sentence in corpus.test]

In [12]:
real_tags = [sentence.tag for sentence in sentences]
predicted_tags = []

In [13]:
for i in range(0, len(sentences), 8):
    sents = sentences[i:i+8]
    model.predict(sents, mini_batch_size=8)
    for sent in sents:
        predicted_tags.append(sent.tag)

In [18]:
for index in range(len(predicted_tags)):
    if predicted_tags[index] != real_tags[index]:
        print(index)
        print(real_tags[index], predicted_tags[index])

259
NON_FOOD_GOODS FOOD_GOODS


In [19]:
corpus.test[259]

Sentence[15]: "Оплата за Уход за одеждой и обувью по счету 11837472833255495630 от 21.06.2024г. Сумма 4110-00" → NON_FOOD_GOODS (1.0)

In [5]:
print(result.detailed_results)


Results:
- F-score (micro) 0.998
- F-score (macro) 0.9988
- Accuracy 0.998

By class:
                precision    recall  f1-score   support

NON_FOOD_GOODS     1.0000    0.9896    0.9948        96
    FOOD_GOODS     0.9890    1.0000    0.9945        90
       SERVICE     1.0000    1.0000    1.0000        88
  BANK_SERVICE     1.0000    1.0000    1.0000        49
           TAX     1.0000    1.0000    1.0000        48
          LOAN     1.0000    1.0000    1.0000        41
       LEASING     1.0000    1.0000    1.0000        38
   REALE_STATE     1.0000    1.0000    1.0000        27
NOT_CLASSIFIED     1.0000    1.0000    1.0000        23

      accuracy                         0.9980       500
     macro avg     0.9988    0.9988    0.9988       500
  weighted avg     0.9980    0.9980    0.9980       500



In [1]:
GPT, BERT 
GPT != BERT -> вручную

SyntaxError: invalid syntax (43358497.py, line 2)

In [ ]:
1) Квантизация 8
2) оннх
3) пайторч тредс